## Import Required Libraries

In [ ]:
%pip install --q -r requirements.txt

In [ ]:
from nba_api.stats.endpoints import teamdetails, commonteamyears, teamgamelogs,playergamelogs, commonplayerinfo, boxscoreadvancedv3, boxscoretraditionalv3, boxscoremiscv3, scoreboardv2
import pandas as pd
from tqdm import tqdm
import os
import pandavro as pdx
import numpy as np
import time
from datetime import datetime, date
from warnings import filterwarnings
import json
import multiprocessing as mp
filterwarnings('ignore')

## Get All Team IDs

In [ ]:
if 'all_teams.parquet' not in os.listdir('Static Files'):
    teams = commonteamyears.CommonTeamYears()
    print('Established connection')
    teams = teams.get_dict()
    print('Retrieved list of dicts')
    team_ids = []
    for team in tqdm(teams['resultSets'][0]['rowSet']):
        team_id = team[1]
        team_name = team[2]
        start_year = team[3]
        end_year = team[4]
        team_ids.append((team_id, team_name, start_year, end_year))
        time.sleep(0.5)
    print('Established list of tuples')
    teams = pd.DataFrame(team_ids, columns=['team_id', 'year_founded', 'year_depreciated', 'abbreviation'])
    print('Created DataFrame')
    teams.to_parquet('Static Files/all_teams.parquet', index=False)

## Get All Team Details

In [ ]:
if 'team_details.parquet' not in os.listdir('Static Files'):
    teams = pd.read_parquet('Static Files/all_teams.parquet')
    initial_df = []
    for team in tqdm(teams['team_id']):
        get_coaches = teamdetails.TeamDetails(team_id=team, timeout=60)
        next_df = get_coaches.get_data_frames()[0]
        initial_df.append(next_df)
        time.sleep(0.5)
    output_df = pd.concat(initial_df)
    output_df.to_parquet('Static Files/team_details.parquet')

## Get All Team Game Logs

In [ ]:
if 'team_game_logs.parquet' not in os.listdir('Static Files'):
    start = 0
    end = 1
    initial_df = []
    for integer in tqdm(range(1,25)):
        if end < 10:
            get_game_stats = teamgamelogs.TeamGameLogs(season_nullable=f'200{start}-0{end}')
            next_df = get_game_stats.get_data_frames()[0]
            initial_df.append(next_df)
            time.sleep(0.5)
            start += 1
            end += 1
        else:
            start += 1
            end += 1
            get_game_stats = teamgamelogs.TeamGameLogs(season_nullable=f'20{start}-{end}')
            next_df = get_game_stats.get_data_frames()[0]
            initial_df.append(next_df)
            time.sleep(0.5)

    output_df = pd.concat(initial_df)
    output_df.to_parquet('Static Files/team_game_logs.parquet')

## Get All Current Player Injury Reports

In [ ]:
if 'player_injury_reports.avro' not in os.listdir('Static Files'):
    player_injuries = pd.concat(pd.read_html('https://www.espn.com/nba/injuries')).fillna('No Info Available')
    player_injuries['date_updated'] = datetime.today()
    pdx.to_avro('Static Files/player_injury_reports.avro', player_injuries)
elif ('player_injury_reports.avro' in os.listdir('Static Files')):
    player_injuries = pdx.read_avro('Static Files/player_injury_reports.avro')
    if player_injuries['date_updated'].all() != datetime.today():
        player_injuries = pd.concat(pd.read_html('https://www.espn.com/nba/injuries')).fillna('No Info Available')
        player_injuries['date_updated'] = datetime.today()
        pdx.to_avro('Static Files/player_injury_reports.avro', player_injuries)

## Get All Player Game Logs

In [ ]:
if 'player_game_logs.parquet' not in os.listdir('Static Files'):
    start = 0
    end = 1
    initial_df = []
    for integer in tqdm(range(1,25)):
        if end < 10:
            get_game_stats = playergamelogs.PlayerGameLogs(season_nullable=f'200{start}-0{end}')
            next_df = get_game_stats.get_data_frames()[0]
            initial_df.append(next_df)
            time.sleep(0.5)
            start += 1
            end += 1
        else:
            start += 1
            end += 1
            get_game_stats = playergamelogs.PlayerGameLogs(season_nullable=f'20{start}-{end}')
            next_df = get_game_stats.get_data_frames()[0]
            initial_df.append(next_df)
            time.sleep(0.5)

    output_df = pd.concat(initial_df)
    output_df.to_parquet('Static Files/player_game_logs.parquet')

## Get All Games Next Season

In [ ]:
if '2024-25_game_ids.parquet' not in os.listdir('Static Files'):
    start = datetime.strptime("2024-10-04", "%Y-%m-%d")
    end = datetime.strptime("2025-04-15", "%Y-%m-%d")

    date_range = pd.date_range(start=start, end=end).to_list()

    games = {}
    # Assuming 'range' contains the list of dates for the 2024-25 season
    for date in tqdm(date_range):
        json_string = json.loads(scoreboardv2.ScoreboardV2(game_date=date).get_json())

        i = 0  # resultSets index for 'GameHeader'
        headers = json_string['resultSets'][i].get('headers', '')

        # Iterate through each game in the 'rowSet'
        for game in json_string['resultSets'][i]['rowSet']:
            game_id = game[2]  # Index 2 is the 'GAME_ID'

            # Add game_id to the list of games for the specific date
            if date in games:
                games[date].append(game_id)
            else:
                games[date] = [game_id]

        time.sleep(0.5)  # Delay to avoid rate limits
    games = {key.strftime('%Y-%m-%d'): value for key, value in games.items()}
    games_2025 = pd.DataFrame(list(games.items()), columns=['game_date', 'game_id'])
    games_2025 = games_2025.explode(column='game_id')
    games_2025.to_parquet('Static Files/2024-25_game_ids.parquet')

## Get Advanced Player Game Stats

In [ ]:
if 'current_player_advanced_game_stats.parquet' not in os.listdir('Static Files'):
    # Filter games for the 2023-24 season
    games = game_log_df[game_log_df['SEASON_YEAR'] == '2023-24']['GAME_ID'].unique()
    rows = []

    # Iterate over each game
    for game_id in tqdm(games):
        try:
            # Fetch box score data for the current game
            game = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id).get_data_frames()[0]

            # Extract relevant fields and append rows
            rows.extend([{
                'game_id': game_id,
                'team_id': player_stats.teamId,
                'team_name': f"{player_stats.teamCity} {player_stats.teamName}",
                'player_id': player_stats.personId,
                'player_name': f"{player_stats.firstName} {player_stats.familyName}",
                'position': player_stats.position,
                'comment': player_stats.comment,
                'offensive_rating': player_stats.offensiveRating,
                'defensive_rating': player_stats.defensiveRating,
                'net_rating': player_stats.netRating,
                'assist_percentage': player_stats.assistPercentage,
                'assist_turnover_ratio': player_stats.assistToTurnover,
                'assist_ratio': player_stats.assistRatio,
                'usage_percentage': player_stats.usagePercentage,
                'possessions': player_stats.possessions,
                'effective_fg_percentage': player_stats.effectiveFieldGoalPercentage,
                'true_shooting_percentage': player_stats.trueShootingPercentage,
                'player_impact_score': player_stats.PIE
            } for player_stats in game.itertuples()])

            # Sleep to avoid hitting the rate limit
            time.sleep(0.5)
        except Exception as e:
            print(f"Error processing game {game_id}: {e}")
            continue

    # Convert list of rows to DataFrame
    df = pd.DataFrame(rows)

    # Save to Parquet
    df.to_parquet('Static Files/current_player_advanced_game_stats.parquet')


## Get Advanced Team Game Stats

In [ ]:
def fetch_team_advanced_stats():

    if 'current_team_advanced_game_stats.parquet' not in os.listdir('Static Files'):
        # Filter games for the 2023-24 season
        games = game_log_df2[game_log_df2['SEASON_YEAR'] == '2023-24']['GAME_ID'].unique()
        rows = []

        # Iterate over each game
        for game_id in tqdm(games):
            try:
                # Fetch box score data for the current game
                game = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id).get_data_frames()[1]

                # Extract relevant fields and append rows
                rows.extend([{
                    'game_id': game_id,
                    'team_id': team_stats.teamId,
                    'team_name': f"{team_stats.teamCity} {team_stats.teamName}",
                    'offensive_rating': team_stats.offensiveRating,
                    'defensive_rating': team_stats.defensiveRating,
                    'net_rating': team_stats.netRating,
                    'assist_percentage': team_stats.assistPercentage,
                    'assist_turnover_ratio': team_stats.assistToTurnover,
                    'assist_ratio': team_stats.assistRatio,
                    'possessions': team_stats.possessions,
                    'effective_fg_percentage': team_stats.effectiveFieldGoalPercentage,
                    'true_shooting_percentage': team_stats.trueShootingPercentage
                } for team_stats in game.itertuples()])

                # Sleep to avoid hitting the rate limit
                time.sleep(1)
            except:
                print(f"last processed game: {game_id}")
                df = pd.DataFrame(rows)
                df.to_parquet('Static Files/current_team_advanced_game_stats.parquet')

        # Convert list of rows to DataFrame
        df = pd.DataFrame(rows)

        # Save to Parquet
        df.to_parquet('Static Files/current_team_advanced_game_stats.parquet')

# Run the function
fetch_team_advanced_stats()


## Get Player Stats

In [53]:
from nba_api.live.nba.endpoints import boxscore
import json
import pandas as pd

# get home player stats
home_players = json.loads(boxscore.BoxScore(game_id='0012400027').get_json())['game']['homeTeam']['players']
home_team = json.loads(boxscore.BoxScore(game_id='0012400027').get_json())['game']['homeTeam']['teamTricode']
home_player_stats = []
for player in home_players:
    stats = player['statistics']
    stats['player_name'] = f"{player['firstName']} {player['familyName']}"
    stats['team_name'] = home_team  # Include team name

    home_player_stats.append(stats)
home_player_stats = pd.DataFrame(home_player_stats)
home_player_stats = home_player_stats[['team_name', 'player_name'] + [col for col in home_player_stats.columns if col not in ['team_name', 'player_name']]]

# get away player stats
away_players = json.loads(boxscore.BoxScore(game_id='0012400027').get_json())['game']['awayTeam']['players']
away_team = json.loads(boxscore.BoxScore(game_id='0012400027').get_json())['game']['awayTeam']['teamTricode']
away_player_stats = []
for player in away_players:
    stats = player['statistics']
    stats['player_name'] = f"{player['firstName']} {player['familyName']}"
    stats['team_name'] = away_team  # Include team name

    away_player_stats.append(stats)
away_player_stats = pd.DataFrame(away_player_stats)
away_player_stats = away_player_stats[['team_name', 'player_name'] + [col for col in away_player_stats.columns if col not in ['team_name', 'player_name']]]
home_player_stats = home_player_stats[['team_name', 'player_name', 'minutes', 'points', 'reboundsTotal', 'assists', 'steals', 'blocks', 'blocksReceived', 'turnovers', 'foulsPersonal', 'foulsDrawn', 'foulsTechnical', 'fieldGoalsAttempted', 'fieldGoalsMade', 'fieldGoalsPercentage', 'freeThrowsMade', 'freeThrowsAttempted', 'freeThrowsPercentage', 'threePointersAttempted', 'threePointersMade', 'threePointersPercentage', 'pointsFastBreak', 'pointsInThePaint', 'pointsSecondChance']]
home_player_stats.rename(columns={'reboundsTotal': 'rebounds', 'blocksReceived': 'blocks_against', 'fieldGoalsAttempted': 'field_goals_attempted', 'fieldGoalsMade': 'field_goals_made', 'freeThrowsAttempted': 'free_throws_attempted', 'threePointersAttempted': 'three_pointers_attempted', 'foulsPersonal': 'personal_fouls', 'foulsDrawn': 'fouls_drawn', 'foulsTechnical': 'technical_fouls', 'fieldGoalsPercentage': 'field_goals_percentage', 'freeThrowsPercentage': 'free_throws_percentage', 'threePointersPercentage': 'three_pointers_percentage', 'freeThrowsMade': 'free_throws_made', 'threePointersMade': 'three_pointers_made', 'pointsFastBreak': 'fast_break_points', 'pointsInThePaint': 'points_in_the_paint', 'pointsSecondChance': 'second_chance_points'}, inplace=True)

home_player_stats['minutes'] = home_player_stats['minutes'].str.replace('PT', '')
home_player_stats['minutes'] = home_player_stats['minutes'].str.replace('M', ':')
home_player_stats['minutes'] = home_player_stats['minutes'].str.replace('S', '')
home_player_stats

,team_name,player_name,minutes,points,rebounds,assists,steals,blocks,blocks_against,turnovers,...,field_goals_percentage,free_throws_made,free_throws_attempted,free_throws_percentage,three_pointers_attempted,three_pointers_made,three_pointers_percentage,fast_break_points,points_in_the_paint,second_chance_points
0,MEM,John Konchar,23:17.00,8,5,1,1,1,0,0,...,0.333333,2,2,1.00,5,2,0.400000,3,0,3
1,MEM,Santi Aldama,24:53.00,10,4,5,2,0,0,4,...,0.428571,1,2,0.50,7,3,0.428571,3,0,3
2,MEM,Zach Edey,22:24.00,10,5,0,0,1,1,1,...,0.250000,6,8,0.75,3,0,0.000000,2,4,1
3,MEM,Desmond Bane,24:47.00,17,2,3,2,0,1,2,...,0.538462,0,0,0.00,5,3,0.600000,0,8,0
4,MEM,Marcus Smart,15:13.00,4,1,0,2,0,0,2,...,0.500000,2,2,1.00,1,0,0.000000,0,2,0
5,MEM,Scotty Pippen Jr.,20:32.00,9,4,5,1,0,1,3,...,0.600000,3,4,0.75,1,0,0.000000,2,4,0
6,MEM,Jake LaRavia,14:12.00,3,3,3,1,0,0,2,...,0.166667,1,2,0.50,2,0,0.000000,0,2,2
7,MEM,Jay Huff,23:11.00,15,6,2,1,3,0,0,...,0.500000,1,2,0.50,6,4,0.666667,6,2,5
8,MEM,Jaylen Wells,25:08.00,12,6,3,2,0,2,4,...,0.333333,3,3,1.00,4,1,0.250000,0,6,2
9,MEM,Luke Kennard,18:41.00,5,2,0,0,0,0,0,...,0.285714,0,0,0.00,4,1,0.250000,3,2,0
